In [2]:
import os
import sys
import numpy as np
import json

sys.path.append('..')
from configs.arguments import get_config_dict
from utils.multiview_utils import Camera, Calibration, MultiviewVids
from utils.io_utils import write_json, load_json
from utils.metadata_utils import get_cam_names
from utils.coordinate_utils import update_reconstruction, point_in_polygon, project_to_ground_plane_cv2
from utils.plot_utils import rotation_matrix, perp

from scipy.spatial.transform import Rotation as R
from skspatial.objects import Point, Vector, Plane, Points, Line
import pyransac3d as pyrsc

import copy
import cv2
import ipywidgets as widgets
import ipympl
import matplotlib.pyplot as plt
%matplotlib widget

arv_copy = sys.argv
sys.argv = ['pop']
sys.argv.append('-cfg')
config_path = os.path.abspath('../../project_config.yaml')
sys.argv.append(config_path)
sys.argv.append('-dr')
root_path = os.path.abspath('../../data/')
sys.argv.append(root_path)
sys.argv.append('-l')
sys.argv.append('info')

# load arguments from the arg parser
config = get_config_dict()
data_root = config["main"]["data_root"]
calib_dir = os.path.join(data_root, '0-calibration', 'calibs')
video_dir = os.path.join(data_root, 'raw_data', 'footage')
reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'undistorted', 'reconstruction.json')
omni_tag = '360'


reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'reconstruction.json')
if not os.path.exists(reconstruction_dir):
    print( "Reconstruction not found. Please run 0-calibration/2-extrinsics.py first.")
    

10:16:32.336 - [...calibration\..\configs\arguments.py:093] - DEBUG    - No training config file specified, using default config in data_root/2-training/train_config.yaml


In [3]:
mvvids = MultiviewVids(newest=False, config=config)

max_frame = np.min([10, mvvids.get_max_frame_id() - 1])
step = 2
base_frames = {}

frame_ids = list(np.arange(0, max_frame, step))

base_frames = mvvids.extract_mv(frame_ids, undistort = True)

# Load reconstruction
reconstruction = load_json(reconstruction_dir)[0]

10:16:32.374 - [...bration\..\utils\multiview_utils.py:859] - INFO     - Cameras: ['cam1', 'cam2', 'cam3', 'cam4']
10:16:32.392 - [...bration\..\utils\multiview_utils.py:867] - INFO     - Loading calibration data for camera 'cam1'
10:16:32.407 - [...bration\..\utils\multiview_utils.py:867] - INFO     - Loading calibration data for camera 'cam2'
10:16:32.428 - [...bration\..\utils\multiview_utils.py:261] - WARNING  - Discarding frame IDs {0} that are out of range.
10:16:32.429 - [...bration\..\utils\multiview_utils.py:647] - WARNING  - Unable to read JSON f:\Github\MARMOT\data\0-calibration\calibs\cam4.json, initialising with default values
10:16:32.429 - [...bration\..\utils\multiview_utils.py:649] - WARNING  - list index out of range


In [58]:
# from utils.plot_utils import plot_3d_reconstruction
plt.close('all')
import cv2
import copy

import numpy as np
import pyransac3d as pyrsc
import matplotlib.pyplot as plt
import ipywidgets as widgets

from typing import List, Tuple, Dict, Union, TypedDict, Optional
from scipy.spatial.transform import Rotation as R
from skspatial.objects import Points, Plane, Line, Vector, Point
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from utils.io_utils import write_json
from utils.coordinate_utils import update_reconstruction, point_in_polygon
from utils.multiview_utils import Camera, MultiviewVids
from utils.plot_utils import rotation_matrix


mvvids = copy.deepcopy(mvvids)
plot_recon = []
for i in reconstruction['points'].keys():
    plot_recon.append(reconstruction['points'][i]['coordinates'])

# remove outliers from the reconstruction
plot_recon_new = np.array(plot_recon)

# detect outliers by computing the distance from the mean
dist = np.linalg.norm(plot_recon - np.mean(plot_recon, axis=0), axis=1)
plot_recon_new = plot_recon_new[dist < 2 * np.std(dist)]

camera_centers = []
for camera in mvvids.cams:
    camera.calib_from_reconstruction(reconstruction = reconstruction)
    camera_centers.append(camera.get_position())
    H = camera.get_ground_plane_homography(input_img_size = (1920, 1080), output_img_size = (1920, 1080))
    
temp_reconstruction = copy.deepcopy(reconstruction)


# pip install 'jupyterlab>=3' ipywidgets 'pyvista[all,trame]'

import pyvista as pv
import numpy as np

curr_normal = [0,0,1]
curr_origin = [0,0,0]
# Create a plotter object
plotter = pv.Plotter(shape=(1,2))

# Add your objects to the plotter
# For example, create and add a plane
# plane = pv.Plane(center=(0, 0, 0), direction=(0, 0, 1))

sample = np.random.choice(plot_recon_new.shape[0], 
                          500, replace=False)

point_cloud = pv.PolyData(plot_recon_new[sample])
camera_locations = pv.PolyData(camera_centers)

plotter.subplot(0,0)

plotter.add_mesh(point_cloud, color='blue', point_size=5)
plotter.add_mesh(camera_locations, color='red', point_size=10)
imgs = [cv2.resize(base_frames[camera.name][0], camera.calibration.size) for camera in mvvids.cams]

def callback(normal, origin):
    global curr_normal, curr_origin
    curr_normal = normal
    curr_origin = origin
    plotter.subplot(0,1)

    rotation = R.from_matrix(rotation_matrix(curr_normal, [0,0,1]))

    temp_reconstruction = update_reconstruction(copy.deepcopy(reconstruction), rotation = rotation, origin=curr_origin, scaling = 1)

    part_img = []

    for i, cam in enumerate(mvvids.cams):
        cam.calib_from_reconstruction(reconstruction = temp_reconstruction)
        H = cam.get_ground_plane_homography(input_img_size = cam.calibration.size, output_img_size = (1920, 1080))

        part_img.append(project_to_ground_plane_cv2(imgs[i], H, (1920, 1080)))
    
    # new_img = np.zeros((1920, 1080, 3), dtype=np.uint8)
    new_img = None
    for img_ in part_img:
        # convert to RGB
        img_ = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)
        if isinstance(new_img, np.ndarray):
            new_img = cv2.addWeighted(new_img, 0.5, img_, 0.5, 0)
        else:
            new_img = img_
    plotter.subplot(0,1)

    # Create a plane to which the texture will be mapped
    plane = pv.Plane(direction=(0, 0, 1), i_size=new_img.shape[1], j_size=new_img.shape[0])


    texture = pv.numpy_to_texture(new_img)
    plotter.add_mesh(plane, texture = texture)

def key_press_event(vtk_obj, event):
    global curr_normal, curr_origin
    if vtk_obj.GetKeyCode() == 'r':
        curr_normal = [0,0,1]
        curr_origin = [0,0,0]
        callback(curr_normal, curr_origin)
    elif vtk_obj.GetKeyCode() == 's':
        print('Saving...')
        write_json(os.path.join(data_root, '0-calibration', 'opensfm', 'undistorted', 'reconstruction_test.json'), temp_reconstruction)
        print('Saved!')
    
plotter.subplot(0,1)
plotter.camera_position = [(0, 0, 2000), (0, 0, 0), (0, 2000, 0)]
plotter.enable_camera_rotation = False
plotter.enable_zoom = False
plotter.enable_pan = False

plotter.subplot(0,0)     
plotter.add_plane_widget(callback, normal=[0,0,1])


# Show the plotter
plotter.show()






Widget(value='<iframe src="http://localhost:53733/index.html?ui=P_0x218b5ec1f70_46&reconnect=auto" class="pyvi…

In [ ]:
plt.close('all')

from matplotlib.patches import Polygon


roi_points = {cam.name: [] for cam in mvvids.cams}

# create dropdown widget with camera names as options
cam_dropdown = widgets.Dropdown(
    options=[cam.name for cam in mvvids.cams],
    value=mvvids.cams[0].name,
    description='Camera:',
    disabled=False,
)

def onclick(event, cam):
    ax.plot(event.xdata, event.ydata, 'rx', markersize=10)
    roi_points[cam.name].append(([event.xdata, event.ydata]))

    # check if we have at least 3 points to define a polygon
    if len(roi_points[cam.name]) >= 3:
        # get the x and y coordinates of the clicked points
        x, y = zip(*roi_points[cam.name])

        # create a polygon patch with the clicked points
        polygon = Polygon(list(zip(x, y)), alpha=0.2, facecolor='C0')

        # remove any existing polygon patches from the axes
        for patch in ax.patches:
            patch.remove()

        # add the new polygon patch to the axes
        ax.add_patch(polygon)

    fig.canvas.draw()
    pass

def update_cam(change):
    ax.cla()
    global cam
    cam_name = change.new
    cam = next(cam for cam in mvvids.cams if cam.name == cam_name)
    ax.set_title(f"Camera: {cam.name}")
    ax.imshow(base_frames[cam.name][0])
    fig.canvas.draw()
    pass

cam_dropdown.observe(update_cam, names='value')

cam = mvvids.cams[0]
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(base_frames[cam.name][0])
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

cid = fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, cam))

display(cam_dropdown)
# display(fig)

In [ ]:
# Select 2 points on the ground in the image to define the scale

known_distance = 500.0 # distance between two points in cm

cam = mvvids.cams[0]
frame_num = len(base_frames[cam.name]) - 1
img = copy.deepcopy(base_frames[cam.name][frame_num])

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.set_title(f"Select 2 points on the ground in the image to define the scale")
ax.imshow(img)
ax.axis('off')

cam_center = cam.get_position()
first_point = None
second_point = None
scale_info = {}
distance = known_distance
def onclick3(event):
    global first_point
    global second_point
    global distance
    global scale_info
    x, y = event.xdata, event.ydata
    # img = cv2.drawMarker(img1, (int(x), int(y)), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2)

    if first_point is None:
        # find the 3D point on the ground plane
        first_point = (x, y)
        ax.plot(x, y, 'rx')
    elif second_point is None:
        second_point = (x, y)
        ax.plot(x, y, 'bx')
    
    scale_info = {'id': cam.calibration.view_id,'distance': known_distance, 
              'point1': first_point, 'point2': second_point}

    ax.imshow(img)
    fig.canvas.draw()



cid = fig.canvas.mpl_connect('button_press_event', onclick3)

In [ ]:
from pathlib import Path
# Save updated reconstruction
# new_reconstruction = update_reconstruction(copy.deepcopy(reconstruction), rotation = rotation, origin=origin, scaling = scale)
new_reconstruction_path = os.path.join(data_root, '0-calibration', 'opensfm', 'reconstruction_new.json')
write_json(new_reconstruction_path, [temp_reconstruction])

# Save updated calibrations
for camera in mvvids.cams:
    print(f"Saving calibration data for camera: {camera.name}")
    camera.calib_from_reconstruction(reconstruction = temp_reconstruction)
    camera.calib_path =  Path(data_root) / '0-calibration' / 'calibs' / f'{camera.name}_new.json'
    camera.calibration = camera.calibration._replace(ROI = roi_points[camera.name])
    camera.calibration = camera.calibration._replace(bounding_box = rect)
    camera.save_calibration(calibration = camera.calibration)